In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import requests
import json
import xml.etree.ElementTree as ET
from tqdm import tqdm

# 5.Data Collection with API

## 5.2 Dart

url에 key와 value값을 추가하여 호출

site: https://opendart.fss.or.kr/ <br>
reference: https://opendart.fss.or.kr/guide/main.do?apiGrpCd=DS001 <br>

In [1]:
API_KEY = '6a7ae5ce3e4037d4f907b4eba7cf496ea6cbeb83'

### 공시정보

```python
https://opendart.fss.or.kr/api/document.xml
```

In [3]:
tree = ET.parse('./data/CORPCODE.xml')
root = tree.getroot()

In [5]:
corpcode = pd.DataFrame()
for i, child in enumerate(tqdm(root)):
    data = {
        child[0].tag: child[0].text,
        child[1].tag: child[1].text,
        child[2].tag: child[2].text,
        child[3].tag: child[3].text,
    }

    corpcode = pd.concat([corpcode, pd.DataFrame(data, index=[i])])

100%|██████████| 100519/100519 [11:36<00:00, 144.36it/s]


In [7]:
corpcode.query('stock_code != " "')

,corp_code,corp_name,stock_code,modify_date
2008,00260985,한빛네트,036720,20170630
2020,00264529,엔플렉스,040130,20170630
2021,00358545,동서정보기술,055000,20170630
2783,00231567,애드모바일,032600,20170630
3886,00247939,씨모스,037600,20170630
...,...,...,...,...
100470,00296078,APS,054620,20230426
100471,01118643,로보쓰리에이아이앤로보틱스,238500,20230426
100472,00624518,다산솔루에타,154040,20230426
100473,01065679,알비케이그룹,215790,20230426


In [36]:
corpcode.to_csv(
    './data/corpcode.csv',
    index=False,
    encoding='utf-8-sig'
)

### 기업개황

```python
https://opendart.fss.or.kr/api/company.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380
```

In [17]:
corp_code = '00126380'
info_url = f'https://opendart.fss.or.kr/api/company.json?crtfc_key={API_KEY}&corp_code={corp_code}'
response = requests.get(info_url)
json.loads(response.text)

{'status': '000',
 'message': '정상',
 'corp_code': '00126380',
 'corp_name': '삼성전자(주)',
 'corp_name_eng': 'SAMSUNG ELECTRONICS CO,.LTD',
 'stock_name': '삼성전자',
 'stock_code': '005930',
 'ceo_nm': '한종희, 경계현',
 'corp_cls': 'Y',
 'jurir_no': '1301110006246',
 'bizr_no': '1248100998',
 'adres': '경기도 수원시 영통구  삼성로 129 (매탄동)',
 'hm_url': 'www.samsung.com/sec',
 'ir_url': '',
 'phn_no': '02-2255-0114',
 'fax_no': '031-200-7538',
 'induty_code': '264',
 'est_dt': '19690113',
 'acc_mt': '12'}

### 공시검색

```
https://opendart.fss.or.kr/api/list.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&bgn_de=20200117&end_de=20200117&corp_cls=Y&page_no=1&page_count=10
```

In [18]:
disclosure_url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de=20230504&end_de=20230504&corp_cls=Y&page_no=1&page_count=100'
response = requests.get(disclosure_url)
response = json.loads(response.text)

In [21]:
disclosures = pd.DataFrame()
for page in range(1, 4):
    disclosure_url = f'https://opendart.fss.or.kr/api/list.json?crtfc_key={API_KEY}&bgn_de=20230504&end_de=20230504&corp_cls=Y&page_no={page}&page_count=100'
    response = requests.get(disclosure_url)
    response = json.loads(response.text).get('list')

    disclosures = pd.concat([disclosures, pd.DataFrame(response)])

In [22]:
disclosures

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00146542,일정실업,008500,Y,기타시장안내(개선기간 부여),20230504800774,유가증권시장본부,20230504,유
1,00652423,인바이오젠,101140,Y,기타시장안내(개선기간 부여),20230504800776,유가증권시장본부,20230504,유
2,00154426,아이에이치큐,003560,Y,기타시장안내(개선기간 부여),20230504800777,유가증권시장본부,20230504,유
3,00585219,세원이앤씨,091090,Y,기타시장안내(개선기간 부여),20230504800770,유가증권시장본부,20230504,유
4,00349097,케이티스카이라이프,053210,Y,기업설명회(IR)개최(안내공시),20230504800779,케이티스카이라이프,20230504,유
...,...,...,...,...,...,...,...,...,...
17,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000787,카카오,20230504,
18,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000786,카카오,20230504,
19,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000785,카카오,20230504,
20,00258801,카카오,035720,Y,[기재정정]주식매수선택권부여에관한신고,20230503000778,카카오,20230504,


### 재무재표

```python
https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key=xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx&corp_code=00126380&bsns_year=2018&reprt_code=11011&fs_div=OFS
```

In [34]:
financial_statement_url = f'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json?crtfc_key={API_KEY}&corp_code=00126380&bsns_year=2022&reprt_code=11011&fs_div=CFS'
response = requests.get(financial_statement_url)
response = json.loads(response.text)
pd.DataFrame(response.get('list'))

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,bfefrmtrm_nm,bfefrmtrm_amount,ord,currency,thstrm_add_amount
0,20230307000542,11011,2022,00126380,BS,재무상태표,ifrs-full_CurrentAssets,유동자산,-,제 54 기,218470581000000,제 53 기,218163185000000,제 52 기,198215579000000,1,KRW,NaN
1,20230307000542,11011,2022,00126380,BS,재무상태표,ifrs-full_CashAndCashEquivalents,현금및현금성자산,-,제 54 기,49680710000000,제 53 기,39031415000000,제 52 기,29382578000000,2,KRW,NaN
2,20230307000542,11011,2022,00126380,BS,재무상태표,dart_ShortTermDepositsNotClassifiedAsCashEquiv...,단기금융상품,-,제 54 기,65102886000000,제 53 기,81708986000000,제 52 기,92441703000000,3,KRW,NaN
3,20230307000542,11011,2022,00126380,BS,재무상태표,-표준계정코드 미사용-,단기상각후원가금융자산,-,제 54 기,414610000000,제 53 기,3369034000000,제 52 기,2757111000000,4,KRW,NaN
4,20230307000542,11011,2022,00126380,BS,재무상태표,ifrs-full_CurrentFinancialAssetsAtFairValueThr...,단기당기손익-공정가치금융자산,-,제 54 기,29080000000,제 53 기,40757000000,제 52 기,71451000000,5,KRW,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|기타자본항목,제 54 기,1938328000000,제 53 기,-2128473000000,제 52 기,-8687155000000,13,KRW,NaN
181,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|매각예정분류기타자본항목,제 54 기,,제 53 기,0,제 52 기,-12132000000,13,KRW,NaN
182,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|이익잉여금 [member],제 54 기,337946407000000,제 53 기,293064763000000,제 52 기,271068211000000,13,KRW,NaN
183,20230307000542,11011,2022,00126380,SCE,자본변동표,ifrs-full_Equity,기말자본,자본 [member]|지배기업 소유주지분|자본금 [member],제 54 기,897514000000,제 53 기,897514000000,제 52 기,897514000000,13,KRW,NaN


# 6.Various Data Sources

공공데이터포털: https://www.data.go.kr/ <br>
서울열린데이터광장: http://data.seoul.go.kr/ <br>
경제통계: https://ecos.bok.or.kr/ <br>
보건의료빅데이터개방시스템: https://opendata.hira.or.kr/home.do <br>
문화공공데이터광장: https://www.culture.go.kr/data/main/main.do <br>
VWORLD: https://www.vworld.kr/dev/v4api.do <br>
네이버 API: https://developers.naver.com/products/service-api/datalab/datalab.md <br>
카카오 API: https://developers.kakao.com/ <br>
대신증권 API: https://money2.daishin.com/E5/WTS/Customer/GuideTrading/DW_CybosPlus_Page.aspx?m=9508&p=8812&v=8632 <br>
미국공공데이터: https://www.data.gov/ <br>


간곡한 부탁

- 뉴스  
기간: 2023.03.01 ~ 2023.04.30  
중앙일보  
한겨레
- 유튜브API  
호갱구조대: 정말가지가지한다  
백종원: [백종원의 님아 그 시장을 가오_EP.22_천안] 촌놈 백종원에게 지적해 달라길래 도저히 못 참고 한마디 했습니다. “이것보다 맛있으면 반칙이에요!”  
- 다트API  
재무제표  
네이버  
카카오

# 7.DataBase



> DB(DataBase): 구조화된 정보 또는 데이터의 조직화된 모음  
DBMS(DataBase Management System): 사용자와 데이터베이스 사이에서 사용자의 요구에 따라 정보를 생성해 주고 데이터베이스를 관리해 주는 소프트웨어  
RDBMS(Relational DataBase Management System): 테이블과 다른 테이블이 관계를 맺고 모여있는 집합체  
SQL(Structured Query Language): 관계형 데이터베이스에서 사용되는 언어로 데이터를 쿼리, 조작 및 정의하고 액세스 제어를 제공  

## 7.1 PostgreSQL

오픈소스 RMDBS의 한 종류

## 7.2 기본 커맨드
접속
```
psql -U postgres
```
<br>

종료
```
\q
```
<br>

DB 조회
```
\l or \list  
```
<br>

유저 조회
```
\du
```  
<br>

테이블 조회
```
\dt(+)
```
<br>

테이블 정보 조회
```
\d *table_name*
```  
<br>

테이블 접근
 ```
 \c *table_name*
 ```

## 7.3 테이블 커맨드

생성
```
CREATE TABLE *table_name*  
(  
  id SERIAL PRIMARY KEY,  
  press_name VARCHAR(10) UNIQUE NOT NULL  
);
```
<br>

삭제
```
DROP TABLE *table_name*;
```
<br>

컬럼 추가
```
ALTER TABLE *table_name* ADD COLUMN *column* varchar(10);
```
<br>

컬럼 삭제
```
ALTER TABLE *table_name* DROP COLUMN *column*;
```
<br>

컬럼 이름 변경
```
ALTER TABLE *table_name* RENAME COLUMN *current_column* TO *new_column*;
```
<br>

컬럼 타입 변경
```
ALTER TABLE *table_name* ALTER COLUMN *column* TYPE *data_type*;
```

## 7.4 CRUD

Create<br>
테이블 전체 컬럼 순서대로 데이터 입력
```
INSERT INTO *table_name* 
VALUES (
  *value1*,
  *value2*,
  ...
)
```
<br>

테이블 특정 컬럼 데이터 입력
```
INSERT INTO *table_name*
(
  *column1*,
  *column2*
)
VALUES
(
  *value1*,
  *value2*
);
```
<br>

Read
```
SELECT * FROM *table_name* # 모든 컬럼 조회
SELECT *column_name1, column_name2, ...* FROM *table_name* # 특정 컬럼 조회
```
<br>

Update<br>
일반적으로는 where절과 같이 사용
```
UPDATE *table_name* SET *column_name* = *value*
```
<br>

Delete<br>
일반적으로는 where절과 같이 사용
```
DROP FROM *table_name* # 전체 테이블 내용 삭제
```
<br>


## 7.5 EDA

### 새로운 컬럼 만들기

```
SELECT DISTINCT *column*, (*column2* + *column3*)/2 as avg
FROM *table_name*;
```

### 중복 제거
```
SELECT DISTINCT *column*
FROM *table_name*;
```

### 조건식

#### WHERE
단순 조건식
```
SELECT * 
FROM
  *table_name* 
WHERE 
  column != -1;
```
<br>

여러 조건식
```
SELECT * 
FROM 
  *table_name* 
WHERE 
  *column1* != -1 and *column2* >= 10 or *column3* < 100;
```
<br>

조건식 범위 지정 방법
```
WHERE 
  *column* BETWEEN 0 AND 10;
WHERE 
  *column* NOT BETWEEN 0 AND 10;

WHERE 
  *column* IN (0, 1, 10);
WHERE 
  *column* NOT IN (0, 1, 10);
```
<br>

문자열 검색
_: 몇 글자인지 정해줌 <br>
%: 몇 글자인지 정해주지 않음 <br>
```
WHERE 
  *column* LIKE *pattern*;
WHERE 
  *column* NOT LIKE *pattern;
```
<br>

패턴의 종류는 아래와 같음

```
LIKE '200%': 200으로 시작하는 값
LIKE '%200': 200으로 끝나는 값
LIKE '%200%': 200이 포함되는 값
LIKE '_0_': 가운데가 0이 들어가는 3자리 값
LIKE '_1': x1로 끝나는 값
LIKE '_200%': 200 앞에는 어떠한 문자도 상관 없으며 200이 포함된 문자열

```

#### CASE WHEN
조건에 따라 다른 값을 보여줄 때 사용
```
CASE 
  WHEN 
    condition_1 THEN result_1
  WHEN 
    condition_2 THEN result_2
  ...
  END AS *new_column name*

```

### 집계함수

count: row의 개수 반환
```
SELECT count(*) FROM *table_name*;
```
<br>

sum: 합계 반환
```
SELECT sum(*column*) FROM *table_name*;
```
<br>

avg: 평균 반환
```
SELECT avg(*column*) FROM *table_name*;
```
<br>

max: 최댓값 반환
```
SELECT max(*column*) FROM *table_name*;
```
<br>

min: 최솟값 반환
```
SELECT min(*column*) FROM *table_name*;
```

<br>
<br>
<br>

주의: 집계함수는 where 절에 나올 수 없음
```
SELECT id FROM news WHERE news_count = max(news_count);
```
<br>

위의 쿼리는 아래와 같이 수정해야 함
```
SELECT id FROM news
WHERE news_count = (SELECT max(news_count) FROM news);
```

### 정렬

컬럼을 기준으로 오름차순 혹은 내림차순으로 정렬

오름차순
```
SELECT 
  *column*
FROM 
  *table_name*
ORDER BY 
  *column*;
```
<br>

내림차순
```
SELECT 
  *column*
FROM 
  *table_name*
ORDER BY 
  *column* DESC;
```
<br>
<br>

GROUP BY 뒤에는 집계 함수도 들어갈 수 있음
```
SELECT 
  *column*
FROM 
  *table_name*
ORDER BY 
  sum(*column*) DESC;
```

### GROUP BY
행을 특정 그룹으로 묶음

```
SELECT 
  *column*
FROM 
  *table_name*
GROUP BY
	*column*;
```

### RANK

RANK: 전체 순위를 집계
```
SELECT id, press_name, news_count,
       RANK() OVER (PARTITION BY press_name ORDER BY news_count DESC)
FROM news;
```
<br>

DENSE_RANK: 집합 내 순위를 집계 <br>
중복 발생 시 다음 순위를 생략하지 않음 <br>
(1, 1, 2)
```
SELECT id, press_name, news_count,
       DENSE_RANK() OVER (PARTITION BY press_name ORDER BY news_count DESC)
FROM news;
```

### JOIN

(INNER)JOIN<br>
같은 값이 있는 행만 반환
```
SELECT *column1*, *column2*
FROM press_ids as p
JOIN news as n
ON *p.column1* = *n.column1*';
```

LEFT JOIN<br>
왼쪽 테이블에 오른쪽 테이블을 매칭, 값이 없으면 NULL 삽입
```
SELECT *column1*, *column2*
FROM press_ids as p
LEFT JOIN news as n
ON *p.column1* = *n.column1*';
```
<br>

RIGHT JOIN<br>
오른쪽 테이블에 오른쪽 테이블을 매칭, 값이 없으면 NULL 삽입
```
SELECT *column1*, *column2*
FROM press_ids as p
RIGHT JOIN news as n
ON *p.column1* = *n.column1*';
```
<br>

FULL JOIN
오른쪽 테이블과 왼쪽 테이블 매칭
```
SELECT *column1*, *column2*
FROM press_ids as p
RIGHT JOIN news as n
ON *p.column1* = *n.column1*';
```
<br>

CROSS JOIN: Table1과 Table2의 모든 행을 JOIN (별도의 키를 지정하지 않음)
```
SELECT *column1*, *column2*
FROM press_ids as p
CROSS JOIN news as n;
```

## 7.6 Database Link in Python

### psycopg2

설치
```cmd
pip install psycopg2
pip install psycopg2-binary #위의 설치 에러 시 이 코드 실행
```
<br>

사용법
```python
conn = psycopg2.connect(
  host='address',
  dbname='database name',
  user='user name',
  password='password',
  port=port

cur = conn.cursor()
```

CRUD
```python
cur.execute(f'INSERT INTO test (id, press_name) VALUES ({id}, {press_name});')
conn.commit() # CREATE

cur.execute('SELECT * FROM test;')
result_one = cur.fetchone() # READ
result_many = cur.fetchmany() 
result_all = cur.fetchall() 

cur.execute(f'UPDATE test SET press_name={press_name} WHERE id > 5')
conn.commit() # UPDATE

cur.execute('DELETE FROM test WHERE press_name LIKE %조선%;')
conn.commit() # DELETE
```
<br>

pandas 내의 method를 통해서도 사용 가능
```python
pd.read_sql('SELECT * FROM test', conn)
```
<br>

Closer <br>
사용 후 연결 해제 <br>
```python
cur.close()
conn.close()
```
<br>

아래와 같이 사용 가능
```python
with conn.cursor() as cur:
  cur.execute(query)

conn.close()

with psycopg2.connect():
  with conn.cursor() as cur:
    cur.execute(query)
```

### sqlalchemy

설치
```cmd
pip install sqlalchemy
```
<br>

사용법
```python
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Text, String, DateTime

db_url = f'postgres+psycopg2://{USERNAME}:{PASSWORD}@{DB_HOST}:{PORT}/{DB_NAME}'
engine = create_engine(db_url, echo=True)

test.to_sql(
  'schema',
  engine,
  if_exists='append', # replace: 덮어쓰기
  index=False,
  chunksize=5000,
  dtypes={
    'id': Integer,
    'press_name': Text
  }
)
```